# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Desgrange  ->  C. Desgrange  |  ['C. Desgrange']
R. Burn  ->  R. Burn  |  ['R. Burn']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
M. Smith  ->  M. Smith  |  ['M. Smith']
C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']


C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']


Arxiv has 71 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.06035


extracting tarball to tmp_2310.06035...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figures/deprojected-zoom-2
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure figures/deprojected-zoom-2 as tmp_2310.06501/./figures/deprojected-zoom-2.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 98 bibliographic references in tmp_2310.06501/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2310.06551


extracting tarball to tmp_2310.06551...

 done.


Found 24 bibliographic references in tmp_2310.06551/SifCfFpr.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06035-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06035) | **Planetary system architectures with low-mass inner planets: Direct  imaging exploration of mature systems beyond 1 au**  |
|| <mark>C. Desgrange</mark>, et al. -- incl., <mark>R. Burn</mark>, <mark>E. Matthews</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *49 pages including 31 pages of appendices and references, 31 figures, A&A, accepted*|
|**Abstract**| The discovery of planets orbiting at less than 1 au from their host star and less massive than Saturn in various exoplanetary systems revolutionized our theories of planetary formation. The fundamental question is whether these close-in low-mass planets could have formed in the inner disk interior to 1 au, or whether they formed further out in the planet-forming disk and migrated inward. Exploring the role of additional giant planets in these systems may help us to pinpoint their global formation and evolution. We searched for additional substellar companions by using direct imaging in systems known to host close-in small planets. The use of direct imaging complemented by radial velocity and astrometric detection limits enabled us to explore the giant planet and brown dwarf demographics around these hosts to investigate the potential connection between both populations. We carried out a direct imaging survey with VLT/SPHERE to look for outer giant planets and brown dwarf companions in 27 systems hosting close-in low-mass planets discovered by radial velocity. Our sample is composed of very nearby (<20pc) planetary systems, orbiting G-, K-, and M-type mature (0.5-10Gyr) stellar hosts. We performed homogeneous direct imaging data reduction and analysis to search for and characterize point sources, and derived robust statistical detection limits. Of 337 point-source detections, we do not find any new bound companions. We recovered the emblematic very cool T-type brown dwarf GJ229B. Our typical sensitivities in direct imaging range from 5 to 30 MJup beyond 2 au. The non-detection of massive companions is consistent with predictions based on models of planet formation by core accretion. Our pilot study opens the way to a multi-technique approach for the exploration of very nearby exoplanetary systems with future ground-based and space observatories. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06051-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06051) | **Gaia Focused Product Release: Radial velocity time series of long-period  variables**  |
|| G. Collaboration, et al. -- incl., <mark>M. Smith</mark>, <mark>C. Bailer-Jones</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *36 pages, 38 figures*|
|**Abstract**| The third Gaia Data Release (DR3) provided photometric time series of more than 2 million long-period variable (LPV) candidates. Anticipating the publication of full radial-velocity (RV) in DR4, this Focused Product Release (FPR) provides RV time series for a selection of LPVs with high-quality observations. We describe the production and content of the Gaia catalog of LPV RV time series, and the methods used to compute variability parameters published in the Gaia FPR. Starting from the DR3 LPVs catalog, we applied filters to construct a sample of sources with high-quality RV measurements. We modeled their RV and photometric time series to derive their periods and amplitudes, and further refined the sample by requiring compatibility between the RV period and at least one of the $G$, $G_{\rm BP}$, or $G_{\rm RP}$ photometric periods. The catalog includes RV time series and variability parameters for 9\,614 sources in the magnitude range $6\lesssim G/{\rm mag}\lesssim 14$, including a flagged top-quality subsample of 6\,093 stars whose RV periods are fully compatible with the values derived from the $G$, $G_{\rm BP}$, and $G_{\rm RP}$ photometric time series. The RV time series contain a mean of 24 measurements per source taken unevenly over a duration of about three years. We identify the great most sources (88%) as genuine LPVs, with about half of them showing a pulsation period and the other half displaying a long secondary period. The remaining 12% consists of candidate ellipsoidal binaries. Quality checks against RVs available in the literature show excellent agreement. We provide illustrative examples and cautionary remarks. The publication of RV time series for almost 10\,000 LPVs constitutes, by far, the largest such database available to date in the literature. The availability of simultaneous photometric measurements gives a unique added value to the Gaia catalog (abridged) |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06175-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06175) | **Gaia Focused Product Release: Spatial distribution of two diffuse  interstellar bands**  |
|| M. Schultheis, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *29 pages, accepted for publication in A&A*|
|**Abstract**| Diffuse interstellar bands (DIBs) are absorption features seen in optical and infrared spectra of stars that are probably caused by large and complex molecules in the ISM. Here we investigate the Galactic distribution and properties of two DIBs identified in almost six million stellar spectra collected by the Gaia Radial Velocity Spectrometer. These measurements constitute a part of the Gaia Focused Product Release to be made public between the Gaia DR3 and DR4 data releases. In order to isolate the DIB signal from the stellar features in each individual spectrum, we identified a set of 160 000 spectra at high Galactic latitudes which we consider to be the DIB-free reference sample. Matching each target spectrum to its closest reference spectra in stellar parameter space allowed us to remove the stellar spectrum empirically, without reference to stellar models, leaving a set of six million ISM spectra. Identifying the two DIBs at 862.1 nm and 864.8 nm in the stacked spectra, we modelled their shapes and report the depth, central wavelength, width, and equivalent width (EW) for each, along with confidence bounds on these measurements. Our main results are as follows: (1) the strength and spatial distribution of the DIB $\lambda$862.1 are very consistent with what was found in Gaia DR3, but for this work we attained a higher signal-to-noise ratio in the stacked spectra to larger distances, which allowed us to trace DIBs in the outer spiral arm and beyond the Scutum--Centaurus spiral arm; (2) we produced an all-sky map below ${\pm}65^{\circ}$ of Galactic latitude to $\sim$4000 pc of both DIB features and their correlations; (3) we detected the signals of DIB\,$\lambda$862.1 inside the Local Bubble; and (4) there is a reasonable correlation with the dust reddening found from stellar absorption and EWs of both DIBs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06295-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06295) | **Gaia Focused Product Release: A catalogue of sources around quasars to  search for strongly lensed quasars**  |
|| G. Collaboration, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *35 pages, 60 figures, accepted for publication by Astronomy and Astrophysics*|
|**Abstract**| Context. Strongly lensed quasars are fundamental sources for cosmology. The Gaia space mission covers the entire sky with the unprecedented resolution of $0.18$" in the optical, making it an ideal instrument to search for gravitational lenses down to the limiting magnitude of 21. Nevertheless, the previous Gaia Data Releases are known to be incomplete for small angular separations such as those expected for most lenses. Aims. We present the Data Processing and Analysis Consortium GravLens pipeline, which was built to analyse all Gaia detections around quasars and to cluster them into sources, thus producing a catalogue of secondary sources around each quasar. We analysed the resulting catalogue to produce scores that indicate source configurations that are compatible with strongly lensed quasars. Methods. GravLens uses the DBSCAN unsupervised clustering algorithm to detect sources around quasars. The resulting catalogue of multiplets is then analysed with several methods to identify potential gravitational lenses. We developed and applied an outlier scoring method, a comparison between the average BP and RP spectra of the components, and we also used an extremely randomised tree algorithm. These methods produce scores to identify the most probable configurations and to establish a list of lens candidates. Results. We analysed the environment of 3 760 032 quasars. A total of 4 760 920 sources, including the quasars, were found within 6" of the quasar positions. This list is given in the Gaia archive. In 87\% of cases, the quasar remains a single source, and in 501 385 cases neighbouring sources were detected. We propose a list of 381 lensed candidates, of which we identified 49 as the most promising. Beyond these candidates, the associate tables in this Focused Product Release allow the entire community to explore the unique Gaia data for strong lensing studies further. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06501-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06501) | **A sensitive, high-resolution, wide-field IRAM NOEMA CO(1-0) survey of  the very nearby spiral galaxy IC 342**  |
|| M. Querejeta, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark>, <mark>K. Kreckel</mark>, <mark>N. Neumayer</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *16 pages, 6 figures, accepted for publication in A&A*|
|**Abstract**| We present a new wide-field 10.75 x 10.75 arcmin^2 (~11x11 kpc^2), high-resolution (theta = 3.6" ~ 60 pc) NOEMA CO(1-0) survey of the very nearby (d=3.45 Mpc) spiral galaxy IC 342. The survey spans out to about 1.5 effective radii and covers most of the region where molecular gas dominates the cold interstellar medium. We resolved the CO emission into >600 individual giant molecular clouds and associations. We assessed their properties and found that overall the clouds show approximate virial balance, with typical virial parameters of alpha_vir=1-2. The typical surface density and line width of molecular gas increase from the inter-arm region to the arm and bar region, and they reach their highest values in the inner kiloparsec of the galaxy (median Sigma_mol~80, 140, 160, and 1100 M_sun/pc^2, sigma_CO~6.6, 7.6, 9.7, and 18.4 km/s for inter-arm, arm, bar, and center clouds, respectively). Clouds in the central part of the galaxy show an enhanced line width relative to their surface densities and evidence of additional sources of dynamical broadening. All of these results agree well with studies of clouds in more distant galaxies at a similar physical resolution. Leveraging our measurements to estimate the density and gravitational free-fall time at 90 pc resolution, averaged on 1.5 kpc hexagonal apertures, we estimate a typical star formation efficiency per free-fall time of 0.45% with a 16-84% variation of 0.33-0.71% among such 1.5 kpc regions. We speculate that bar-driven gas inflow could explain the large gas concentration in the central kiloparsec and the buildup of the massive nuclear star cluster. This wide-area CO map of the closest face-on massive spiral galaxy demonstrates the current mapping power of NOEMA and has many potential applications. The data and products are publicly available. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06551-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06551) | **Gaia Focused Product Release: Sources from Service Interface Function  image analysis -- Half a million new sources in omega Centauri**  |
|| G. Collaboration, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| **|
|**Abstract**| Gaia's readout window strategy is challenged by very dense fields in the sky. Therefore, in addition to standard Gaia observations, full Sky Mapper (SM) images were recorded for nine selected regions in the sky. A new software pipeline exploits these Service Interface Function (SIF) images of crowded fields (CFs), making use of the availability of the full two-dimensional (2D) information. This new pipeline produced half a million additional Gaia sources in the region of the omega Centauri ($\omega$ Cen) cluster, which are published with this Focused Product Release. We discuss the dedicated SIF CF data reduction pipeline, validate its data products, and introduce their Gaia archive table. Our aim is to improve the completeness of the {\it Gaia} source inventory in a very dense region in the sky, $\omega$ Cen. An adapted version of {\it Gaia}'s Source Detection and Image Parameter Determination software located sources in the 2D SIF CF images. We validated the results by comparing them to the public {\it Gaia} DR3 catalogue and external Hubble Space Telescope data. With this Focused Product Release, 526\,587 new sources have been added to the {\it Gaia} catalogue in $\omega$ Cen. Apart from positions and brightnesses, the additional catalogue contains parallaxes and proper motions, but no meaningful colour information. While SIF CF source parameters generally have a lower precision than nominal {\it Gaia} sources, in the cluster centre they increase the depth of the combined catalogue by three magnitudes and improve the source density by a factor of ten. This first SIF CF data publication already adds great value to the {\it Gaia} catalogue. It demonstrates what to expect for the fourth {\it Gaia} catalogue, which will contain additional sources for all nine SIF CF regions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06216-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06216) | **An Extreme FeLoBAL Outflow in the VLT/UVES Spectrum of Quasar SDSS  J1321-0041**  |
|| D. Byun, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *6 pages, 5 figures, 2 tables. Submitted to A&A*|
|**Abstract**| Context. Quasar outflows are often analyzed to determine their ability to contribute to active galactic nucleus (AGN) feedback. We have identified a broad absorption line (BAL) outflow in the VLT/UVES spectrum of the quasar SDSS J1321-0041. The outflow shows troughs from Fe II, and is therefore identified as an FeLoBAL. It is quite extreme among that population, as it shows C II and Si II BALs. Aims. Outflow systems require a kinetic luminosity above $\sim0.5\%$ of the quasar's luminosity to contribute to AGN feedback. For this reason, we analyzed the spectrum of J1321-0041 to determine the outflow's kinetic luminosity, as well as the quasar's bolometric luminosity. Methods. We measured the ionic column densities from the absorption troughs in the spectrum, and determined the Hydrogen column density and ionization parameter using those column densities as our constraints. We also determined the electron number density $n_e$ based on the ratios between excited state and resonance state column densities of Fe II and Si II. This allowed us to find the distance of the outflow from its central source, as well as its kinetic luminosity. Results. We determined the kinetic luminosity of the outflow to be $8.4^{+13.2}_{-5.3}\times 10^{45}\text{ erg s}^{-1}$, and the quasar's bolometric luminosity to be $1.72\pm0.13\times10^{47}\text{ erg s}^{-1}$, resulting in a ratio of $\dot{E}_k/L_{Bol}=4.8^{+7.7}_{-3.1}\%$. We conclude that this outflow has sufficient kinetic luminosity to contribute to AGN feedback. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06463-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06463) | **IAS/CEA Evolution of Dust in Nearby Galaxies (ICED): the  spatially-resolved dust properties of NGC4254**  |
|| L. Pantoni, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-10-11*|
|*Comments*| *to appear in Proc. of the mm Universe 2023 conference, Grenoble (France), June 2023, published by F. Mayet et al. (Eds), EPJ Web of conferences, EDP Sciences*|
|**Abstract**| We present the first preliminary results of the project \textit{ICED}, focusing on the face-on galaxy NGC4254. We use the millimetre maps observed with NIKA2 at IRAM-30m, as part of the IMEGIN Guaranteed Time Large Program, and of a wide collection of ancillary data (multi-wavelength photometry and gas phase spectral lines) that are publicly available. We derive the global and local properties of interstellar dust grains through infrared-to-radio spectral energy distribution fitting, using the hierarchical Bayesian code HerBIE, which includes the grain properties of the state-of-the-art dust model, THEMIS. Our method allows us to get the following dust parameters: dust mass, average interstellar radiation field, and fraction of small grains. Also, it is effective in retrieving the intrinsic correlations between dust parameters and interstellar medium properties. We find an evident anti-correlation between the interstellar radiation field and the fraction of small grains in the centre of NGC4254, meaning that, at strong radiation field intensities, very small amorphous carbon grains are efficiently destroyed by the ultra-violet photons coming from newly formed stars, through photo-desorption and sublimation. We observe a flattening of the anti-correlation at larger radial distances, which may be driven by the steep metallicity gradient measured in NGC4254. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.06035.md
    + _build/html/tmp_2310.06035/./fig/target_indiv/PMD_GJ422_2epochs_zoom.png
    + _build/html/tmp_2310.06035/./fig/detlim_contrast_curves/theory_vs_obs_DIonly_M.png
    + _build/html/tmp_2310.06035/./fig/detlim_contrast_curves/theory_vs_obs_DIonly_GK.png
    + _build/html/tmp_2310.06035/./fig/mosaics/mosaic_IRDIS_SpeCal_cADI_paper.png
exported in  _build/html/2310.06051.md
    + _build/html/tmp_2310.06051/./drv_dg_2panels.png
    + _build/html/tmp_2310.06051/./RVTS_ARCep_cf_alvarez01_1.png
    + _build/html/tmp_2310.06051/./RVTS_RSCrB_cf_hinkle02_1.png
    + _build/html/tmp_2310.06051/./RVTS_RNor_cf_lebzelter05_1.png
    + _build/html/tmp_2310.06051/./RVTS_ex_mixPph.png
exported in  _build/html/2310.06175.md
    + _build/html/tmp_2310.06175/./fit-example.png
    + _build/html/tmp_2310.06175/./Corner_0.png
    + _build/html/tmp_2310.06175/./DIB-width.png
exported in  _build/html/2310.06295.md
    + _build/html/tmp_2310.06295/./Figures/G2237+0305

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dd}{\textnormal{d}}$
$\newcommand{\pa}[1]{\left(#1\right)}$
$\newcommand{\pac}[1]{\left[#1\right]}$
$\newcommand{\paa}[1]{\left\{#1\right\}}$
$\newcommand{\teff}{\ensuremath{\mathrm{T_{eff}}}}$
$\newcommand{\logg}{\ensuremath{\log(g)}}$
$\newcommand{\logZ}{\ensuremath{\log(Z/Z_{\odot})}}$
$\newcommand{\pc}{\ensuremath{\mathrm{pc}}}$
$\newcommand{\magg}{\ensuremath{\mathrm{mag}}}$
$\newcommand{\kelvin}{\ensuremath{\mathrm{K}}}$
$\newcommand{\mas}{\ensuremath{\mathrm{mas}}}$
$\newcommand{\yr}{\ensuremath{\mathrm{year}}}$
$\newcommand{\yrs}{\ensuremath{\mathrm{years}}}$
$\newcommand{\monthh}{\ensuremath{\mathrm{month}}}$
$\newcommand{\months}{\ensuremath{\mathrm{months}}}$
$\newcommand{\dayy}{\ensuremath{\mathrm{day}}}$
$\newcommand{\days}{\ensuremath{\mathrm{days}}}$
$\newcommand{\hr}{\ensuremath{\mathrm{hour}}}$
$\newcommand{\hrs}{\ensuremath{\mathrm{hours}}}$
$\newcommand{\khz}{\ensuremath{\mathrm{kHz}}}$
$\newcommand{\hz}{\ensuremath{\mathrm{Hz}}}$
$\newcommand{\myr}{\ensuremath{\mathrm{Myr}}}$
$\newcommand{\gyr}{\ensuremath{\mathrm{Gyr}}}$
$\newcommand{\au}{\ensuremath{\mathrm{au}}}$
$\newcommand{\mic}{\ensuremath{\mathrm{\mu m}}}$
$\newcommand{\nm}{\ensuremath{\mathrm{nm}}}$
$\newcommand{\pic}{\ensuremath{\mathrm{pm}}}$
$\newcommand{\mj}{\ensuremath{\mathrm{M_{Jup}}}}$
$\newcommand{\rj}{\ensuremath{\mathrm{R_{Jup}}}}$
$\newcommand{\mearth}{\ensuremath{\mathrm{M}_\oplus}}$
$\newcommand{\rearth}{\ensuremath{\textnormal{R}_{\oplus}}}$
$\newcommand{\densityearth}{\ensuremath{\rho_{\oplus}}}$
$\newcommand{\msun}{\ensuremath{\textnormal{M}_\odot}}$
$\newcommand{\rsun}{\ensuremath{\textnormal{R}_\odot}}$
$\newcommand{\lsun}{\ensuremath{\textnormal{L}_\odot}}$
$\newcommand{◦}{\ensuremath{^\circ}}$
$\newcommandinecolor{lagoon}{rgb}{0.5, 0.7, 0.8}$
$\newcommandinecolor{forest green}{rgb}{0.233, 0.645, 0.233}$
$\newcommand{\edc}[1]{\textcolor{lagoon}{#1}}$
$\newcommand{\celia}[1]{\textcolor{lagoon}{ [\textbf{Célia: } #1]}}$
$\newcommand{\gael}[1]{\textcolor{magenta}{ [\textbf{Gaël: } #1]}}$
$\newcommand{\JM}[1]{\textcolor{forest green}{ #1 [JM]}}$
$\setlength{\LTcapwidth}{\textwidth}$
$\setlength{\tabcolsep}{5pt}$
$\begin{document}$
$   \title{Planetary system architectures with low-mass inner planets$
$}$
$\subtitle{Direct imaging\thanks{Based on observations collected at the European Southern Observatory under ESO programmes 099.C-0255(A) 102.C-0489(A), 103.C-0484 (A and B), and 109.23F2.} exploration of mature systems beyond 1 au}$
$   \authorrunning{Desgrange et al.}$
$   \author{C.~Desgrange\inst{1}\fnmsep\inst{2}, J.~Milli\inst{1}, G.~Chauvin\inst{3}, Th.~Henning\inst{2}, A.~Luashvili\inst{4}, M.~Read\inst{5}, M.~Wyatt\inst{6}, G.~Kennedy\inst{7}\fnmsep\inst{8}, R.~Burn\inst{2}, M.~Schlecker\inst{9}, F.~Kiefer\inst{10}, V.~D'Orazi\inst{11}\fnmsep\inst{12}, S.~Messina\inst{13}, P.~Rubini\inst{14}, A.-M.~Lagrange\inst{10}, C.~Babusiaux\inst{1}, L.~Matrà\inst{15}, B.~Bitsch\inst{2}, M.~Bonavita\inst{12}\fnmsep\inst{16}, P.~Delorme\inst{1}, E.~Matthews\inst{2}, P.~Palma-Bifani\inst{3}, A.~Vigan\inst{17}$
$        }$
$   \institute{$
$    ^{1} Univ. Grenoble Alpes, CNRS, IPAG, F-38000 Grenoble, France;  e-mail: celia.desgrange@univ-grenoble-alpes.fr \^{2} Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany\^{3}\;Laboratoire Lagrange, UMR7293, Université  Cote d’Azur, CNRS, Observatoire de la C\^ote d’Azur. Boulevard de l’Observatoire, 06304 Nice, France. \^{4} Laboratoire Univers et Théories, Observatoire de Paris, Université PSL, CNRS, Université Paris Cité, 92190 Meudon, France\^{5} Space Research and Planetology Division, Physikalisches Inst., Universität Bern, Switzerland \^{6} Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom \^{7} Department of Physics, University of Warwick, Gibbet Hill Road, Coventry, CV4 7AL, UK\^{8} Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\^{9} Department of Astronomy/Steward Observatory, The University of Arizona, 933 North Cherry Avenue, Tucson, AZ 85721, USA \^{10}\;LESIA, Observatoire de Paris, Université PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Jules Janssen, 92195 Meudon, France \^{11} Department of Physics, University of Rome Tor Vergata, Via della Ricerca Scientifica 1, 00133 Rome, Italy\^{12} INAF-Osservatorio Astronomico di Padova, Vicolo dell’ Osservatorio 5, 35122, Padova, Italy\^{13} INAF–Osservatorio Astrofisico di Catania, via Santa Sofia, 78 Catania, Italy \^{14} Pixyl, 5 Avenue du Grand Sablon, 38700 La Tronche, France\^{15} School of Physics, Trinity College Dublin, The University of Dublin, College Green, Dublin 2, Ireland\^{16} School of Physical Sciences, The Open University, Walton Hall, Milton Keynes, MK7 6AA, UK\^{17} Aix Marseille Univ, CNRS, CNES, LAM, Marseille, France\}$
$   \date $
$  \abstract{The discovery of planets orbiting at less than 1 au from their host star and less massive than Saturn in various exoplanetary systems revolutionized our theories of planetary formation.$
$   The fundamental question is whether these close-in low-mass planets could have formed in the inner disk interior to 1 au, or whether they formed further out in the planet-forming disk and migrated inward.$
$   Exploring the role of additional giant planet(s) in these systems may help us to pinpoint their global formation and evolution.$
$   }{We searched for additional substellar companions  by using direct imaging in systems known to host close-in small planets. The use of direct imaging complemented by radial velocity and astrometric detection limits enabled us to explore the giant planet and brown dwarf demographics around these hosts to investigate the potential connection between both populations.}{We carried out a direct imaging survey with SPHERE at VLT to look for outer giant planets and brown dwarf companions in 27 systems hosting close-in low-mass planets discovered by radial velocity. Our sample is composed of very nearby (< 20 pc) planetary systems, orbiting G-, K-, and M-type mature (0.5--10 Gyr) stellar hosts. We performed homogeneous direct imaging data reduction and analysis to search for and characterize point sources, and derived robust statistical detection limits.$
$   The final direct imaging detection performances were globally considered together  with radial velocity and astrometric sensitivity. }{Of 337 point-source detections, we do not find any new bound companions. We recovered the emblematic very cool T-type brown dwarf GJ 229 B. Our typical sensitivities in direct imaging range from 5 to 30 M_{\rm Jup} beyond 2 au. The non-detection of massive companions is consistent with predictions based on models of planet formation by core accretion. Our pilot study opens the way to a multi-technique approach for the exploration of very nearby exoplanetary systems with future ground-based and space observatories.}$
$    $
$   \keywords{Planetary systems -- Instrumentation: adaptive optics, high angular resolution -- Methods: observational}$
$   \titlerunning{Architecture of systems hosting close-in sub-Saturns}$
$   \maketitle$
$\bibliographystyle{aa.bst}$
$\n\end{document}}}}$
$\newcommand{\pac}[1]{\left[#1\right]}$
$\newcommand{\paa}[1]{\left\{#1\right\}}$
$\newcommand{\teff}{\ensuremath{\mathrm{T_{eff}}}}$
$\newcommand{\logg}{\ensuremath{\log(g)}}$
$\newcommand{\logZ}{\ensuremath{\log(Z/Z_{\odot})}}$
$\newcommand{\pc}{\ensuremath{\mathrm{pc}}}$
$\newcommand{\magg}{\ensuremath{\mathrm{mag}}}$
$\newcommand{\kelvin}{\ensuremath{\mathrm{K}}}$
$\newcommand{\mas}{\ensuremath{\mathrm{mas}}}$
$\newcommand{\yr}{\ensuremath{\mathrm{year}}}$
$\newcommand{\yrs}{\ensuremath{\mathrm{years}}}$
$\newcommand{\monthh}{\ensuremath{\mathrm{month}}}$
$\newcommand{\months}{\ensuremath{\mathrm{months}}}$
$\newcommand{\dayy}{\ensuremath{\mathrm{day}}}$
$\newcommand{\days}{\ensuremath{\mathrm{days}}}$
$\newcommand{\hr}{\ensuremath{\mathrm{hour}}}$
$\newcommand{\hrs}{\ensuremath{\mathrm{hours}}}$
$\newcommand{\khz}{\ensuremath{\mathrm{kHz}}}$
$\newcommand{\hz}{\ensuremath{\mathrm{Hz}}}$
$\newcommand{\myr}{\ensuremath{\mathrm{Myr}}}$
$\newcommand{\gyr}{\ensuremath{\mathrm{Gyr}}}$
$\newcommand{\au}{\ensuremath{\mathrm{au}}}$
$\newcommand{\mic}{\ensuremath{\mathrm{\mu m}}}$
$\newcommand{\nm}{\ensuremath{\mathrm{nm}}}$
$\newcommand{\pic}{\ensuremath{\mathrm{pm}}}$
$\newcommand{\mj}{\ensuremath{\mathrm{M_{Jup}}}}$
$\newcommand{\rj}{\ensuremath{\mathrm{R_{Jup}}}}$
$\newcommand{\mearth}{\ensuremath{\mathrm{M}_\oplus}}$
$\newcommand{\rearth}{\ensuremath{\textnormal{R}_{\oplus}}}$
$\newcommand{\densityearth}{\ensuremath{\rho_{\oplus}}}$
$\newcommand{\msun}{\ensuremath{\textnormal{M}_\odot}}$
$\newcommand{\rsun}{\ensuremath{\textnormal{R}_\odot}}$
$\newcommand{\lsun}{\ensuremath{\textnormal{L}_\odot}}$
$\newcommand{◦}{\ensuremath{^\circ}}$
$\newcommandinecolor{lagoon}{rgb}{0.5, 0.7, 0.8}$
$\newcommandinecolor{forest green}{rgb}{0.233, 0.645, 0.233}$
$\newcommand{\edc}[1]{\textcolor{lagoon}{#1}}$
$\newcommand{\celia}[1]{\textcolor{lagoon}{ [\textbf{Célia: } #1]}}$
$\newcommand{\gael}[1]{\textcolor{magenta}{ [\textbf{Gaël: } #1]}}$
$\newcommand{\JM}[1]{\textcolor{forest green}{ #1 [JM]}}$
$\setlength{\LTcapwidth}{\textwidth}$
$\setlength{\tabcolsep}{5pt}$
$\begin{document}$
$   \title{Planetary system architectures with low-mass inner planets$
$}$
$\subtitle{Direct imaging\thanks{Based on observations collected at the European Southern Observatory under ESO programmes 099.C-0255(A) 102.C-0489(A), 103.C-0484 (A and B), and 109.23F2.} exploration of mature systems beyond 1 au}$
$   \authorrunning{Desgrange et al.}$
$   \author{C.~Desgrange\inst{1}\fnmsep\inst{2}, J.~Milli\inst{1}, G.~Chauvin\inst{3}, Th.~Henning\inst{2}, A.~Luashvili\inst{4}, M.~Read\inst{5}, M.~Wyatt\inst{6}, G.~Kennedy\inst{7}\fnmsep\inst{8}, R.~Burn\inst{2}, M.~Schlecker\inst{9}, F.~Kiefer\inst{10}, V.~D'Orazi\inst{11}\fnmsep\inst{12}, S.~Messina\inst{13}, P.~Rubini\inst{14}, A.-M.~Lagrange\inst{10}, C.~Babusiaux\inst{1}, L.~Matrà\inst{15}, B.~Bitsch\inst{2}, M.~Bonavita\inst{12}\fnmsep\inst{16}, P.~Delorme\inst{1}, E.~Matthews\inst{2}, P.~Palma-Bifani\inst{3}, A.~Vigan\inst{17}$
$        }$
$   \institute{$
$    ^{1} Univ. Grenoble Alpes, CNRS, IPAG, F-38000 Grenoble, France;  e-mail: celia.desgrange@univ-grenoble-alpes.fr \^{2} Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany\^{3}\;Laboratoire Lagrange, UMR7293, Université  Cote d’Azur, CNRS, Observatoire de la C\^ote d’Azur. Boulevard de l’Observatoire, 06304 Nice, France. \^{4} Laboratoire Univers et Théories, Observatoire de Paris, Université PSL, CNRS, Université Paris Cité, 92190 Meudon, France\^{5} Space Research and Planetology Division, Physikalisches Inst., Universität Bern, Switzerland \^{6} Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom \^{7} Department of Physics, University of Warwick, Gibbet Hill Road, Coventry, CV4 7AL, UK\^{8} Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\^{9} Department of Astronomy/Steward Observatory, The University of Arizona, 933 North Cherry Avenue, Tucson, AZ 85721, USA \^{10}\;LESIA, Observatoire de Paris, Université PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Jules Janssen, 92195 Meudon, France \^{11} Department of Physics, University of Rome Tor Vergata, Via della Ricerca Scientifica 1, 00133 Rome, Italy\^{12} INAF-Osservatorio Astronomico di Padova, Vicolo dell’ Osservatorio 5, 35122, Padova, Italy\^{13} INAF–Osservatorio Astrofisico di Catania, via Santa Sofia, 78 Catania, Italy \^{14} Pixyl, 5 Avenue du Grand Sablon, 38700 La Tronche, France\^{15} School of Physics, Trinity College Dublin, The University of Dublin, College Green, Dublin 2, Ireland\^{16} School of Physical Sciences, The Open University, Walton Hall, Milton Keynes, MK7 6AA, UK\^{17} Aix Marseille Univ, CNRS, CNES, LAM, Marseille, France\}$
$   \date $
$  \abstract{The discovery of planets orbiting at less than 1 au from their host star and less massive than Saturn in various exoplanetary systems revolutionized our theories of planetary formation.$
$   The fundamental question is whether these close-in low-mass planets could have formed in the inner disk interior to 1 au, or whether they formed further out in the planet-forming disk and migrated inward.$
$   Exploring the role of additional giant planet(s) in these systems may help us to pinpoint their global formation and evolution.$
$   }{We searched for additional substellar companions  by using direct imaging in systems known to host close-in small planets. The use of direct imaging complemented by radial velocity and astrometric detection limits enabled us to explore the giant planet and brown dwarf demographics around these hosts to investigate the potential connection between both populations.}{We carried out a direct imaging survey with SPHERE at VLT to look for outer giant planets and brown dwarf companions in 27 systems hosting close-in low-mass planets discovered by radial velocity. Our sample is composed of very nearby (< 20 pc) planetary systems, orbiting G-, K-, and M-type mature (0.5--10 Gyr) stellar hosts. We performed homogeneous direct imaging data reduction and analysis to search for and characterize point sources, and derived robust statistical detection limits.$
$   The final direct imaging detection performances were globally considered together  with radial velocity and astrometric sensitivity. }{Of 337 point-source detections, we do not find any new bound companions. We recovered the emblematic very cool T-type brown dwarf GJ 229 B. Our typical sensitivities in direct imaging range from 5 to 30 M_{\rm Jup} beyond 2 au. The non-detection of massive companions is consistent with predictions based on models of planet formation by core accretion. Our pilot study opens the way to a multi-technique approach for the exploration of very nearby exoplanetary systems with future ground-based and space observatories.}$
$    $
$   \keywords{Planetary systems -- Instrumentation: adaptive optics, high angular resolution -- Methods: observational}$
$   \titlerunning{Architecture of systems hosting close-in sub-Saturns}$
$   \maketitle$
$\bibliographystyle{aa.bst}$
$\n\end{document}}}$
$\newcommand{\paa}[1]{\left\{#1\right\}}$
$\newcommand{\teff}{\ensuremath{\mathrm{T_{eff}}}}$
$\newcommand{\logg}{\ensuremath{\log(g)}}$
$\newcommand{\logZ}{\ensuremath{\log(Z/Z_{\odot})}}$
$\newcommand{\pc}{\ensuremath{\mathrm{pc}}}$
$\newcommand{\magg}{\ensuremath{\mathrm{mag}}}$
$\newcommand{\kelvin}{\ensuremath{\mathrm{K}}}$
$\newcommand{\mas}{\ensuremath{\mathrm{mas}}}$
$\newcommand{\yr}{\ensuremath{\mathrm{year}}}$
$\newcommand{\yrs}{\ensuremath{\mathrm{years}}}$
$\newcommand{\monthh}{\ensuremath{\mathrm{month}}}$
$\newcommand{\months}{\ensuremath{\mathrm{months}}}$
$\newcommand{\dayy}{\ensuremath{\mathrm{day}}}$
$\newcommand{\days}{\ensuremath{\mathrm{days}}}$
$\newcommand{\hr}{\ensuremath{\mathrm{hour}}}$
$\newcommand{\hrs}{\ensuremath{\mathrm{hours}}}$
$\newcommand{\khz}{\ensuremath{\mathrm{kHz}}}$
$\newcommand{\hz}{\ensuremath{\mathrm{Hz}}}$
$\newcommand{\myr}{\ensuremath{\mathrm{Myr}}}$
$\newcommand{\gyr}{\ensuremath{\mathrm{Gyr}}}$
$\newcommand{\au}{\ensuremath{\mathrm{au}}}$
$\newcommand{\mic}{\ensuremath{\mathrm{\mu m}}}$
$\newcommand{\nm}{\ensuremath{\mathrm{nm}}}$
$\newcommand{\pic}{\ensuremath{\mathrm{pm}}}$
$\newcommand{\mj}{\ensuremath{\mathrm{M_{Jup}}}}$
$\newcommand{\rj}{\ensuremath{\mathrm{R_{Jup}}}}$
$\newcommand{\mearth}{\ensuremath{\mathrm{M}_\oplus}}$
$\newcommand{\rearth}{\ensuremath{\textnormal{R}_{\oplus}}}$
$\newcommand{\densityearth}{\ensuremath{\rho_{\oplus}}}$
$\newcommand{\msun}{\ensuremath{\textnormal{M}_\odot}}$
$\newcommand{\rsun}{\ensuremath{\textnormal{R}_\odot}}$
$\newcommand{\lsun}{\ensuremath{\textnormal{L}_\odot}}$
$\newcommand{◦}{\ensuremath{^\circ}}$
$\newcommandinecolor{lagoon}{rgb}{0.5, 0.7, 0.8}$
$\newcommandinecolor{forest green}{rgb}{0.233, 0.645, 0.233}$
$\newcommand{\edc}[1]{\textcolor{lagoon}{#1}}$
$\newcommand{\celia}[1]{\textcolor{lagoon}{ [\textbf{Célia: } #1]}}$
$\newcommand{\gael}[1]{\textcolor{magenta}{ [\textbf{Gaël: } #1]}}$
$\newcommand{\JM}[1]{\textcolor{forest green}{ #1 [JM]}}$
$\setlength{\LTcapwidth}{\textwidth}$
$\setlength{\tabcolsep}{5pt}$
$\begin{document}$
$   \title{Planetary system architectures with low-mass inner planets$
$}$
$\subtitle{Direct imaging\thanks{Based on observations collected at the European Southern Observatory under ESO programmes 099.C-0255(A) 102.C-0489(A), 103.C-0484 (A and B), and 109.23F2.} exploration of mature systems beyond 1 au}$
$   \authorrunning{Desgrange et al.}$
$   \author{C.~Desgrange\inst{1}\fnmsep\inst{2}, J.~Milli\inst{1}, G.~Chauvin\inst{3}, Th.~Henning\inst{2}, A.~Luashvili\inst{4}, M.~Read\inst{5}, M.~Wyatt\inst{6}, G.~Kennedy\inst{7}\fnmsep\inst{8}, R.~Burn\inst{2}, M.~Schlecker\inst{9}, F.~Kiefer\inst{10}, V.~D'Orazi\inst{11}\fnmsep\inst{12}, S.~Messina\inst{13}, P.~Rubini\inst{14}, A.-M.~Lagrange\inst{10}, C.~Babusiaux\inst{1}, L.~Matrà\inst{15}, B.~Bitsch\inst{2}, M.~Bonavita\inst{12}\fnmsep\inst{16}, P.~Delorme\inst{1}, E.~Matthews\inst{2}, P.~Palma-Bifani\inst{3}, A.~Vigan\inst{17}$
$        }$
$   \institute{$
$    ^{1} Univ. Grenoble Alpes, CNRS, IPAG, F-38000 Grenoble, France;  e-mail: celia.desgrange@univ-grenoble-alpes.fr \^{2} Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany\^{3}\;Laboratoire Lagrange, UMR7293, Université  Cote d’Azur, CNRS, Observatoire de la C\^ote d’Azur. Boulevard de l’Observatoire, 06304 Nice, France. \^{4} Laboratoire Univers et Théories, Observatoire de Paris, Université PSL, CNRS, Université Paris Cité, 92190 Meudon, France\^{5} Space Research and Planetology Division, Physikalisches Inst., Universität Bern, Switzerland \^{6} Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, United Kingdom \^{7} Department of Physics, University of Warwick, Gibbet Hill Road, Coventry, CV4 7AL, UK\^{8} Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\^{9} Department of Astronomy/Steward Observatory, The University of Arizona, 933 North Cherry Avenue, Tucson, AZ 85721, USA \^{10}\;LESIA, Observatoire de Paris, Université PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Jules Janssen, 92195 Meudon, France \^{11} Department of Physics, University of Rome Tor Vergata, Via della Ricerca Scientifica 1, 00133 Rome, Italy\^{12} INAF-Osservatorio Astronomico di Padova, Vicolo dell’ Osservatorio 5, 35122, Padova, Italy\^{13} INAF–Osservatorio Astrofisico di Catania, via Santa Sofia, 78 Catania, Italy \^{14} Pixyl, 5 Avenue du Grand Sablon, 38700 La Tronche, France\^{15} School of Physics, Trinity College Dublin, The University of Dublin, College Green, Dublin 2, Ireland\^{16} School of Physical Sciences, The Open University, Walton Hall, Milton Keynes, MK7 6AA, UK\^{17} Aix Marseille Univ, CNRS, CNES, LAM, Marseille, France\}$
$   \date $
$  \abstract{The discovery of planets orbiting at less than 1 au from their host star and less massive than Saturn in various exoplanetary systems revolutionized our theories of planetary formation.$
$   The fundamental question is whether these close-in low-mass planets could have formed in the inner disk interior to 1 au, or whether they formed further out in the planet-forming disk and migrated inward.$
$   Exploring the role of additional giant planet(s) in these systems may help us to pinpoint their global formation and evolution.$
$   }{We searched for additional substellar companions  by using direct imaging in systems known to host close-in small planets. The use of direct imaging complemented by radial velocity and astrometric detection limits enabled us to explore the giant planet and brown dwarf demographics around these hosts to investigate the potential connection between both populations.}{We carried out a direct imaging survey with SPHERE at VLT to look for outer giant planets and brown dwarf companions in 27 systems hosting close-in low-mass planets discovered by radial velocity. Our sample is composed of very nearby (< 20 pc) planetary systems, orbiting G-, K-, and M-type mature (0.5--10 Gyr) stellar hosts. We performed homogeneous direct imaging data reduction and analysis to search for and characterize point sources, and derived robust statistical detection limits.$
$   The final direct imaging detection performances were globally considered together  with radial velocity and astrometric sensitivity. }{Of 337 point-source detections, we do not find any new bound companions. We recovered the emblematic very cool T-type brown dwarf GJ 229 B. Our typical sensitivities in direct imaging range from 5 to 30 M_{\rm Jup} beyond 2 au. The non-detection of massive companions is consistent with predictions based on models of planet formation by core accretion. Our pilot study opens the way to a multi-technique approach for the exploration of very nearby exoplanetary systems with future ground-based and space observatories.}$
$    $
$   \keywords{Planetary systems -- Instrumentation: adaptive optics, high angular resolution -- Methods: observational}$
$   \titlerunning{Architecture of systems hosting close-in sub-Saturns}$
$   \maketitle$
$\bibliographystyle{aa.bst}$
$\n\end{document}}$
$\newcommand{\teff}{\ensuremath{\mathrm{T_{eff}}}}$
$\newcommand{\logg}{\ensuremath{\log(g)}}$
$\newcommand{\logZ}{\ensuremath{\log(Z/Z_{\odot})}}$
$\newcommand{\pc}{\ensuremath{\mathrm{pc}}}$
$\newcommand{\magg}{\ensuremath{\mathrm{mag}}}$
$\newcommand{\kelvin}{\ensuremath{\mathrm{K}}}$
$\newcommand{\mas}{\ensuremath{\mathrm{mas}}}$
$\newcommand{\yr}{\ensuremath{\mathrm{year}}}$
$\newcommand{\yrs}{\ensuremath{\mathrm{years}}}$
$\newcommand{\monthh}{\ensuremath{\mathrm{month}}}$
$\newcommand{\months}{\ensuremath{\mathrm{months}}}$
$\newcommand{\dayy}{\ensuremath{\mathrm{day}}}$
$\newcommand{\days}{\ensuremath{\mathrm{days}}}$
$\newcommand{\hr}{\ensuremath{\mathrm{hour}}}$
$\newcommand{\hrs}{\ensuremath{\mathrm{hours}}}$
$\newcommand{\khz}{\ensuremath{\mathrm{kHz}}}$
$\newcommand{\hz}{\ensuremath{\mathrm{Hz}}}$
$\newcommand{\myr}{\ensuremath{\mathrm{Myr}}}$
$\newcommand{\gyr}{\ensuremath{\mathrm{Gyr}}}$
$\newcommand{\au}{\ensuremath{\mathrm{au}}}$
$\newcommand{\mic}{\ensuremath{\mathrm{\mu m}}}$
$\newcommand{\nm}{\ensuremath{\mathrm{nm}}}$
$\newcommand{\pic}{\ensuremath{\mathrm{pm}}}$
$\newcommand{\mj}{\ensuremath{\mathrm{M_{Jup}}}}$
$\newcommand{\rj}{\ensuremath{\mathrm{R_{Jup}}}}$
$\newcommand{\mearth}{\ensuremath{\mathrm{M}_\oplus}}$
$\newcommand{\rearth}{\ensuremath{\textnormal{R}_{\oplus}}}$
$\newcommand{\densityearth}{\ensuremath{\rho_{\oplus}}}$
$\newcommand{\msun}{\ensuremath{\textnormal{M}_\odot}}$
$\newcommand{\rsun}{\ensuremath{\textnormal{R}_\odot}}$
$\newcommand{\lsun}{\ensuremath{\textnormal{L}_\odot}}$
$\newcommand{\edc}[1]{\textcolor{lagoon}{#1}}$
$\newcommand{\celia}[1]{\textcolor{lagoon}{ [\textbf{Célia: } #1]}}$
$\newcommand{\gael}[1]{\textcolor{magenta}{ [\textbf{Gaël: } #1]}}$
$\newcommand{\JM}[1]{\textcolor{forest green}{ #1 [JM]}}$
$\newcommand{◦}{\ensuremath{^\circ}}$</div>



<div id="title">

# Planetary system architectures with low-mass inner planets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06035-b31b1b.svg)](https://arxiv.org/abs/2310.06035)<mark>Appeared on: 2023-10-11</mark> -  _49 pages including 31 pages of appendices and references, 31 figures, A&A, accepted_

</div>
<div id="authors">

<mark>C. Desgrange</mark>, et al. -- incl., <mark>R. Burn</mark>, <mark>E. Matthews</mark>

</div>
<div id="abstract">

**Abstract:** The discovery of planets orbiting at less than 1 au from their host star and less massive than Saturn in various exoplanetary systems revolutionized our theories of planetary formation.   The fundamental question is whether these close-in low-mass planets could have formed in the inner disk interior to 1 au, or whether they formed further out in the planet-forming disk and migrated inward.   Exploring the role of additional giant planet(s) in these systems may help us to pinpoint their global formation and evolution. We searched for additional substellar companions  by using direct imaging in systems known to host close-in small planets. The use of direct imaging complemented by radial velocity and astrometric detection limits enabled us to explore the giant planet and brown dwarf demographics around these hosts to investigate the potential connection between both populations. We carried out a direct imaging survey with SPHERE at VLT to look for outer giant planets and brown dwarf companions in 27 systems hosting close-in low-mass planets discovered by radial velocity. Our sample is composed of very nearby (< 20 pc) planetary systems, orbiting G-, K-, and M-type mature (0.5--10 Gyr) stellar hosts. We performed homogeneous direct imaging data reduction and analysis to search for and characterize point sources, and derived robust statistical detection limits.   The final direct imaging detection performances were globally considered together  with radial velocity and astrometric sensitivity. Of 337 point-source detections, we do not find any new bound companions. We recovered the emblematic very cool T-type brown dwarf GJ 229 B. Our typical sensitivities in direct imaging range from 5 to 30 M $_{\rm Jup}$ beyond 2 au. The non-detection of massive companions is consistent with predictions based on models of planet formation by core accretion. Our pilot study opens the way to a multi-technique approach for the exploration of very nearby exoplanetary systems with future ground-based and space observatories.

</div>

<div id="div_fig1">

<img src="tmp_2310.06035/./fig/target_indiv/PMD_GJ422_2epochs_zoom.png" alt="Fig9" width="100%"/>

**Figure 9. -** SPHERE/IRDIS relative astrometry differences of the offset positions of all the point-like sources detected around GJ 422
    in both the second and third epochs (2018-01-28 and 2020-01-18).
    The expected variation of offset positions, if the companion candidates are stationary background objects are shown in blue solid line. The variation is estimated based on the parallactic and proper motions of the primary star. The five companion candidates are clearly identified as background contaminants. At the _bottom-right_ corner, the insert shows a zoom on the location of the candidates.
     (*fig:PMD_GJ422_2+3epochs*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06035/./fig/detlim_contrast_curves/theory_vs_obs_DIonly_M.png" alt="Fig33.1" width="50%"/><img src="tmp_2310.06035/./fig/detlim_contrast_curves/theory_vs_obs_DIonly_GK.png" alt="Fig33.2" width="50%"/>

**Figure 33. -** Detection sensitivity of companions for our direct imaging SPHERE observations of 15 M-stars (_left_) and 12 GK-stars (_right_).  The isocontours give the number of stars from our survey that have the sensitivity to detect companions at a probability of $50\%$.
    The theoretical population of companions formed via the Bern core-accretion models considering $1000$ systems are indicated in circles of colors associated to stellar masses \citep{Burn2021_NGPPS}. These symbols have a black outline if the systems also host close-in ($\leq1 $\au$$) low-mass ($\leq95 $\mearth$$) planets.
     (*fig:Fig4_Vigan_detlim_obs_vs_theory*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06035/./fig/mosaics/mosaic_IRDIS_SpeCal_cADI_paper.png" alt="Fig21" width="100%"/>

**Figure 21. -** 
    Reduced images from SPHERE-IRDIS for the whole survey for SpeCal-cADI reductions. The factor in the bottom left hand corner of each figure indicates the number by which to multiply the color bar to have the corresponding contrast values.
     (*fig:mosaic_super-earths_survey_IRDIS_cADI*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06035"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\STAB}[1]{\begin{tabular}{@ c@ }#1\end{tabular}}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand{\Gaia}{\textit{Gaia}\xspace}$
$\newcommand{\rvflag}{\texttt{flag\_rv}\xspace}$
$\newcommand{\mags}{{\rm mag}}$
$\newcommand{\kms}{{\rm km s}^{-1}}$
$\newcommand{\days}{{\rm days}}$
$\newcommand{\Teff}{T_{\rm eff}}$
$\newcommand{\gbp}{G_{\rm BP}}$
$\newcommand{\grp}{G_{\rm RP}}$
$\newcommand{\grvs}{G_{\rm RVS}}$
$\newcommand{\rv}{V_R}$
$\newcommand{\meanrv}{\overline{\rv}}$
$\newcommand{\medianrv}{\langle\rv\rangle}$
$\newcommand{\zprv}{\rv^0}$
$\newcommand{\pubrv}{\rv^{\rm DR3}}$
$\newcommand{\erv}{\varepsilon_{\rv}}$
$\newcommand{\meanerv}{\overline{\varepsilon}_{\rv}}$
$\newcommand{\pg}{P_{G}}$
$\newcommand{\pbp}{P_{G_{\rm BP}}}$
$\newcommand{\prp}{P_{G_{\rm RP}}}$
$\newcommand{\prv}{P_{\rv}}$
$\newcommand{\pph}{P_{\rm ph}}$
$\newcommand{\durationrv}{\Delta t_{\rv}}$
$\newcommand{\ncycrv}{n_{\rv}^{\rm cyc}}$
$\newcommand{\ampg}{A_G}$
$\newcommand{\amprv}{A_{\rv}}$
$\newcommand{\nobsrv}{N_{\rv}}$
$\newcommand{\nobsrvraw}{N_{\rv}^{\rm raw}}$
$\newcommand{\noutrv}{N_{\rv}^{\rm out}}$
$\newcommand{\snrv}{{\rm SN}_{\rv}}$
$\newcommand{\wbprp}{W_{\scaleto{\rm BP,RP}{4.5pt}}}$
$\newcommand$</div>



<div id="title">

# $\Gaia$ Focused Product Release: Radial velocity time series of long-period variables

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06051-b31b1b.svg)](https://arxiv.org/abs/2310.06051)<mark>Appeared on: 2023-10-11</mark> -  _36 pages, 38 figures_

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>M. Smith</mark>, <mark>C. Bailer-Jones</mark>

</div>
<div id="abstract">

**Abstract:** The third $\Gaia$ Data Release (DR3) provided photometric time series of more than 2 million long-period variable (LPV) candidates. Anticipating the publication of full radial-velocity data planned with Data Release 4, this Focused Product Release (FPR) provides radial-velocity time series for a selection of LPV candidates with high-quality observations. We describe the production and content of the $\Gaia$ catalog of LPV radial-velocity time series, and the methods used to compute the variability parameters published as part of the $\Gaia$ FPR. Starting from the DR3 catalog of LPV candidates, we applied several filters to construct a sample of sources with high-quality radial-velocity measurements. We modeled their radial-velocity and photometric time series to derive their periods and amplitudes, and further refined the sample by requiring compatibility between the radial-velocity period and at least one of the $G$ , $\gbp$ , or $\grp$ photometric periods. The catalog includes radial-velocity time series and variability parameters for 9 614 sources in the magnitude range $6\lesssim G/\mags\lesssim 14$ , including a flagged top-quality subsample of 6 093 stars whose radial-velocity periods are fully compatible with the values derived from the $G$ , $\gbp$ , and $\grp$ photometric time series.    The radial-velocity time series contain a mean of 24 measurements per source taken unevenly over a duration of about three years.    We identify the great majority of the sources (88 \% ) as genuine LPV candidates, with about half of them showing a pulsation period and the other half displaying a long secondary period. The remaining 12 \% of the catalog consists of candidate ellipsoidal binaries. Quality checks against radial velocities available in the literature show excellent agreement. We provide some illustrative examples and cautionary remarks. The publication of radial-velocity time series for almost ten thousand LPV candidates constitutes, by far, the largest such database available to date in the literature. The availability of simultaneous photometric measurements gives a unique added value to the $\Gaia$ catalog.

</div>

<div id="div_fig1">

<img src="tmp_2310.06051/./drv_dg_2panels.png" alt="Fig9" width="100%"/>

**Figure 9. -** Comparison between the semi-amplitudes $\ampg$, $\amprv$ of the best-fit models of the $G$-band and RV time series for the TQS (light gray symbols in the background). The darker symbols indicate sources whose RV period is consistent with the photometric periods in a 1:1 ratio (top panel) or in a 2:1 ratio (bottom panel). The dashed red line corresponds to Eq. \ref{eq:amp_condition_ell}, and the size of each sample is indicated in the legend. (*fig:drv_dg_2panels*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06051/./RVTS_ARCep_cf_alvarez01_1.png" alt="Fig34.1" width="33%"/><img src="tmp_2310.06051/./RVTS_RSCrB_cf_hinkle02_1.png" alt="Fig34.2" width="33%"/><img src="tmp_2310.06051/./RVTS_RNor_cf_lebzelter05_1.png" alt="Fig34.3" width="33%"/>

**Figure 34. -** Similar to Fig. \ref{fig:RVTS_ex_mixPph}, but showing the $\Gaia$ time series for the sources compared with literature as discussed in Sect. \ref{sec:catalog_quality:comparison_with_rv_data_from_literature:multi_epoch_rv_data}(see also Table \ref{tab:table_cf_rvcurves}). The sources displayed from top to bottom are the SRb star $\object${AR Cep}\citep[compared with][]{alvarez_etal_2001}, the binary SRa star $\object${RS CrB}\citep[compared with][]{hinkle_etal_2002}, and the O-rich Mira $\object${R Nor}\citep[compared with][]{lebzelter_etal_2005a}. Literature RV time series are displayed as magenta circles (with arbitrary phase offset) in the panels showing the folded RV curve. In each case, the $\Gaia$ RV period (indicated in the header of each panel) is used for folding. (*fig:RVTS_cf_lit*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06051/./RVTS_ex_mixPph.png" alt="Fig29" width="100%"/>

**Figure 29. -** Example time series for a source with mixed consistency between the photometric and RV time series. This source has $\pg\simeq$\pbp$\simeq$\prv$$, while $\prp \simeq 0.5 $\prv$$. The panels in the top row show the RV data and model, while the photometric data and corresponding models are shown in the panels in the bottom row (in red, green, and blue for the $\grp$, $G$, and $\gbp$ bands, respectively). For visualization purposes, an arbitrary offset is applied to the $\grp$ and $\gbp$ time series. The $\Gaia$ DR3 source ID of this object is indicated in the title, together with the period and semi-amplitude of the best-fit $G$-band and RV time series models. The panels on the right show the four time series folded by the RV period. (*fig:RVTS_ex_mixPph*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06051"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gaia}{\textit{Gaia}\xspace}$
$\newcommand{\gdr}[1]{\textit{Gaia}~DR{#1}\xspace}$
$\newcommand{\dibspec}{DIB-Spec}$
$\newcommand{\gspspec}{GSP-Spec}$
$\newcommand{\gspphot}{GSP-Phot}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand{\metaDgrad}{\partial \meta / \partial D}$
$\newcommand{\metaZgrad}{\partial \meta / \partial Z}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log{g}}$
$\newcommand{\feh}{\rm[Fe/H]}$
$\newcommand{\meta}{{\rm[M/H]}}$
$\newcommand{\afe}{{\rm[\alpha/Fe]}}$
$\newcommand{\vrad}{V_{\rm rad}}$
$\newcommand{\vraderr}{\sigma_{V_{\rm rad}}}$
$\newcommand{\um}{\rm \mu m}$
$\newcommand{\EBV}{E(B {-} V)}$
$\newcommand{\EBPRP}{\rm E(BP {-} RP)}$
$\newcommand{\dibdepth}{\mathcal{D}}$
$\newcommand{\diblambda}{\lambda_{\rm DIB}}$
$\newcommand{\dibwidth}{\sigma_{\rm DIB}}$
$\newcommand{\Tomaz}[1]{{\color{magenta} [Tomaz: #1] \color{black}}}$
$\newcommand{\Mathias}[1]{{\color{blue} [Mathias:#1]}}$
$\newcommand{\hzhao}[1]{{\color{red} [He: #1]}}$
$\newcommand$
$\newcommand{\kms}{ {\rm km s^{-1}}}$
$\newcommand{\d}{{\rm d}}$
$\newcommand{\}{kpc}$
$\newcommand{\rg}{r_{\rm g}}$
$\newcommand{\}{rp}$
$\newcommand{\}{ra}$
$\newcommand{\dex}{ {\rm dex}}$
$\newcommand{\Myr}{ {\rm Myr}}$
$\newcommand{\}{Gyr}$
$\newcommand{\}{url}$</div>



<div id="title">

# $\gaia$ Focused Product Release: Spatial distribution of two diffuse interstellar bands

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06175-b31b1b.svg)](https://arxiv.org/abs/2310.06175)<mark>Appeared on: 2023-10-11</mark> -  _29 pages, accepted for publication in A&A_

</div>
<div id="authors">

M. Schultheis, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** Diffuse interstellar bands (DIBs) are absorption features seen in optical and infrared spectra of stars and extragalactic objectsthat are probably caused by large and complex molecules in the galactic interstellar medium (ISM). Here we investigate the Galacticdistribution and properties of two DIBs identified in almost six million stellar spectra collected by the $\gaia$ Radial VelocitySpectrometer. These measurements constitute a part of the $\gaia$ Focused Product Release to be made public between the $\gaia$ DR3 andDR4 data releases. In order to isolate the DIB signal from the stellar features in each individual spectrum, we identified a set of160 000 spectra at high Galactic latitudes ( $|b| {\geqslant} 65^{\circ}$ ) covering a range of stellar parameters which weconsider to be the DIB-free reference sample. Matching each target spectrum to its closest reference spectra in stellar parameter spaceallowed us to remove the stellar spectrum empirically, without reference to stellar models, leaving a set of six million ISM spectra.Using the star's parallax and sky coordinates, we then allocated each ISM spectrum to a voxel (VOlume piXEL) on a contiguousthree-dimensional grid with an angular size of $1.8^{\circ}$ (level 5 HEALPix) and 29 unequally sized distance bins. Identifying thetwo DIBs at 862.1 nm ( $\lambda$ 862.1) and 864.8 nm ( $\lambda$ 864.8) in the stacked spectra, we modelled their shapes and report thedepth, central wavelength, width, and equivalent width (EW) for each, along with confidence bounds on these measurements. We thenexplored the properties and distributions of these quantities and compared them with similar measurements from other surveys. Our mainresults are as follows: (1) the strength and spatial distribution of the DIB $\lambda$ 862.1 are very consistent with what was found in $\gaia$ DR3,but for this work we attained a higher signal-to-noise ratio in the stacked spectra to larger distances, which allowed us to trace DIBs in the outerspiral arm and beyond the Scutum--Centaurus spiral arm; (2) we produced an all-sky map below ${\pm}65^{\circ}$ of Galactic latitudeto $\sim$ 4000 pc of both DIB features and their correlations; (3) we detected the signals of DIB $\lambda$ 862.1 inside theLocal Bubble ( $\lesssim$ 200 pc); and (4) there is a reasonable correlation with the dust reddening found from stellar absorption and EWsof both DIBs with a correlation coefficient of 0.90 for $\lambda$ 862.1 and 0.77 for $\lambda$ 864.8.

</div>

<div id="div_fig1">

<img src="tmp_2310.06175/./fit-example.png" alt="Fig5" width="100%"/>

**Figure 5. -** Examples of the fits to DIBs $\lambda$862.1 and $\lambda$864.8 in stacked ISM spectra in five voxels in the same
  direction, whose HEALPix number ($I_{\rm pix} {=} 10 450$) and GC $(\ell_c,b_c) {=} (322.43^{\circ},{-}0.44^{\circ})$
  are marked in the top panel. The black and red lines are the ISM spectra and fitted DIB profiles, respectively, normalized
  by the fitted linear continuum. The error bars indicate the flux uncertainties at each pixel. Orange indicates the region
  that was masked during the fittings. The central heliocentric distance ($d_c$), the number of target spectra ($N_{\rm tar}$),
  mean $\EBPRP$, EWs of two DIBs, and the S/N of the stacked ISM spectrum in each voxel are indicated as well. (*fig:stack-fit*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06175/./Corner_0.png" alt="Fig16" width="100%"/>

**Figure 16. -** Corner plot of the DIB fitting in the voxel with $I_{\rm pix}=10450$ and $d_c=1.05$ kpc (the first panel in Fig.
  \ref{fig:stack-fit} from top to bottom). The histograms and scatter plots show the one- and two-dimensional projections of the
  posterior distributions of the fitted parameters. The red squares and lines indicate the best-fit estimates for each parameter
  in the reproduced fitting. And the dashed blue lines mark the fitted parameters in the output table of {$\dibspec$}. (*fig:corner0*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06175/./DIB-width.png" alt="Fig8" width="100%"/>

**Figure 8. -** Distribution of $\dibwidth$ for DIBs $\lambda$862.1 and $\lambda$864.8 (top and bottom, respectively). The dashed red and
green lines correspond to the "best range" and the "secondary range" of $\dibwidth$ defined in Sect. \ref{subsect:qf} for the
DIB QF, respectively. The distribution of the full {$\dibspec$} results (235 428 voxels) is shown in the left panels, while the right
panels show a quality-controlled sample with $\rm S/N {>} 100$ and $\dibdepth{>} 3R_C$(see Sect. \ref{subsect:dib-width}).
The latter criterion implies QF values of 0 or 2. The number of detected DIBs and the percentage after the quality control
is indicated as well. (*fig:dib-width*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06175"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ra}{\ensuremath{\alpha}}$
$\newcommand{\dec}{\ensuremath{\delta}}$
$\newcommand{\glon}{\ensuremath{l}}$
$\newcommand{\glat}{\ensuremath{b}}$
$\newcommand{\parallax}{\ensuremath{\varpi}}$
$\newcommand{\sigparallax}{\ensuremath{\sigma_{\varpi}}}$
$\newcommand{\pmra}{\ensuremath{\mu_{\ra\ast}}}$
$\newcommand{\pmdec}{\ensuremath{\mu_\dec}}$
$\newcommand{\propm}{\ensuremath{\mu}}$
$\newcommand{\ms}{\ensuremath{\textrm{m s}^{-1}}}$
$\newcommand{\kms}{\ensuremath{\textrm{km s}^{-1}}}$
$\newcommand{\mas}{\ensuremath{\textrm{mas}}}$
$\newcommand{\maspyr}{\ensuremath{\textrm{mas yr}^{-1}}}$
$\newcommand{\deg}{\ensuremath{^\circ}}$
$\newcommand{\gmag}{\ensuremath{G}~}$
$\newcommand{\gbp}{\ensuremath{G_{\rm BP}}}$
$\newcommand{\grp}{\ensuremath{G_{\rm RP}}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\blue}{\textcolor{blue}}$
$\newcommand{\orange}{\textcolor{orange}}$
$\newcommand{\green}{\textcolor{OliveGreen}}$
$\newcommand{\brown}{\textcolor{brown}}$
$\newcommand$
$\newcommand$
$\newcommand{\sersic}{Sérsic\xspace}$
$\newcommand{\gaia}{\textit{Gaia}\xspace}$
$\newcommand{\matfont}[1]{\ensuremath{\boldsymbol{\mathsf{#1}}}}$
$\newcommand{\mat}[1]{\matfont{#1}}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand$
$\newcommand{\deg}{\ensuremath{^\circ}}$</div>



<div id="title">

# $\gaia$ Focused Product Release: A catalogue of sources around quasars to search for strongly lensed quasars.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06295-b31b1b.svg)](https://arxiv.org/abs/2310.06295)<mark>Appeared on: 2023-10-11</mark> -  _35 pages, 60 figures, accepted for publication by Astronomy and Astrophysics_

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** Strongly lensed quasars are fundamental sources for cosmology. The $\gaia$ space mission covers the entire sky with the unprecedented resolution of $0.18$ " in the optical, making it an ideal instrument to search for gravitational lenses down to the limiting magnitude of 21. Nevertheless, the previous $\gaia$ Data Releases are known to be incomplete for small angular separations such as those expected for most lenses. We present the Data Processing and Analysis Consortium GravLens pipeline, which was built to analyse all $\gaia$ detections around quasars and to cluster them into sources, thus producing a catalogue of secondary sources around each quasar. We analysed the resulting catalogue to produce scores that indicate source configurations that are compatible with strongly lensed quasars. GravLens uses the DBSCAN unsupervised clustering algorithm to detect sources around quasars. The resulting catalogue of multiplets is then analysed with several methods to identify potential gravitational lenses. We developed and applied an outlier scoring method, a comparison between the average BP and RP spectra of the components, and we also used an extremely randomised tree algorithm. These methods produce scores to identify the most probable configurations and to establish a list of lens candidates. We analysed the environment of 3 760 032 quasars. A total of 4 760 920 sources, including the quasars, were found within 6 $\arcsec$ of the quasar positions. This list is given in the $\gaia$ archive. In 87 \% of cases, the quasar remains a single source, and in 501 385 cases neighbouring sources were detected. We propose a list of 381 lensed candidates, of which we identified 49 as the most promising. Beyond these candidates, the associate tables in this Focused Product Release allow the entire community to explore the unique $\gaia$ data for strong lensing studies further.

</div>

<div id="div_fig1">

<img src="tmp_2310.06295/./Figures/G2237+0305_final_compressed.jpg" alt="Fig11.1" width="50%"/><img src="tmp_2310.06295/./Figures/2MASSJ13102005-1714579_final_compressed.jpg" alt="Fig11.2" width="50%"/>

**Figure 11. -** Pan-STARRS images  ([Chambers, Magnier and Metcalfe 2016]())  of two known gravitational lenses with an indication of GravLens components in black (filled circles) and entry in \gdr3 in red (squares). Left: the Einstein cross (G2237+0305). Right: 2MASSJ13102005-1714579. The central sources in 2MASSJ13102005-1714579 encompasses two lensing galaxies recovered as  GravLens components. (*panstarrs*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06295/./Figures/DR3Gaia113100.075-441959.69.png" alt="Fig9.1" width="25%"/><img src="tmp_2310.06295/./Figures/DR3Gaia235007.548+365434.45.png" alt="Fig9.2" width="25%"/><img src="tmp_2310.06295/./Figures/DR3Gaia034732.982+350248.6.png" alt="Fig9.3" width="25%"/><img src="tmp_2310.06295/./Figures/DR3Gaia082523.532+241524.53.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Examples of known issues. Black dots are the mean positions of the components, red points correspond to individual observations no matter the component and  gray dots are outliers. In (d) a planetary nebula (IC 351) that unduly entered in the quasar catalogue is decomposed by the algorithm into numerous sources. as well as in (d) for the halo of a bright star. (*Fig:clusteringIssues*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06295/./Figures/qso_G.jpg" alt="Fig8.1" width="25%"/><img src="tmp_2310.06295/./Figures/qso_bprp.jpg" alt="Fig8.2" width="25%"/><img src="tmp_2310.06295/./Figures/qso_w1w2.jpg" alt="Fig8.3" width="25%"/><img src="tmp_2310.06295/./Figures/z_milli.jpg" alt="Fig8.4" width="25%"/>

**Figure 8. -** Distributions of
(a): $\gaia$$G$ magnitudes (phot\_g\_mean\_mag) from the \gdr3 gaia\_source table, (b): $G_$\mat$hrm{BP}-G_$\mat$hrm{RP}$ colours (phot\_bp\_mean\_mag -- phot\_rp\_mean\_mag), (c): W1-W2  colours (from catWISE), (d): redshifts (from Milliquas) of the quasars and candidates from the input list. (*qso_G*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06295"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\emm}[1]{\ensuremath{#1}}$
$\newcommand{\emr}[1]{\emm{\mathrm{#1}}}$
$\newcommand{\unit}[1]{\emr{ #1}}$
$\newcommand{\kms}{\unit{km s^{-1}}}$
$\newcommand{\tint}[1][]{\emm{\Delta t_\emr{#1}}}$
$\newcommand{\beam}[1][]{\emm{\theta_\emr{#1}}}$
$\newcommand{\Msun}{\emm{M_\odot}}$
$\newcommand{\Msunperyr}{\emm{M_\odot \emr{yr^{-1}}}}$
$\newcommand{\Msunperyrperkpcsq}{\emm{M_\odot \emr{yr^{-1}} \emr{kpc^{-2}}}}$
$\newcommand{\MJypersr}{\emm{\emr{MJy} \emr{sr^{-1}}}}$
$\newcommand{\ngmc}{637}$
$\newcommand{\hi}{\ion{H}{I}}$
$\newcommand{\FigRGB}{$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.95\linewidth]{figures/ic0342_dss2_vla_noema_mayall_rgrid_small.png}$
$\caption{Molecular, atomic, and ionized gas in IC 342. This is a false-color composite of IC 342 showing molecular gas traced by our new NOEMA \chem{CO}{10} survey (green), along with stellar light from DSS2 (white), atomic gas traced by the 21 cm line using the VLA (blue), and H\alpha emission from ionized gas (red). \mod{The dashed rectangle delimits} the field of view observed with NOEMA and solid ellipses show galactocentric radii at 5, 10, 15, and 20 kpc.}$
$\label{fig:ic342:false-color}$
$\end{figure*}}$
$\newcommand{\FigData}{$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.475\textwidth]{figures/ic342_tpeak.png}$
$\includegraphics[width=0.475\textwidth]{figures/ic342_mom1.png} \\includegraphics[width=0.475\textwidth]{figures/ic342_wide_logtg.png}$
$\includegraphics[width=0.475\textwidth]{figures/ic0342_rprof.png}$
$\caption{{NOEMA survey of \chem{CO}{10} emission from IC 342.} The distribution of the \chem{CO}{10} peak intensity at the native angular resolution (\textit{top left}) and the intensity-weighted mean CO velocity at 90 pc resolution (\textit{top right}) as revealed by our new CO survey. The images show spiral arms, abundant inter-arm emission, and a velocity field that mostly reflects a regularly rotating gas disk. The \textit{bottom} panels show the molecular gas traced by CO in the context of atomic gas and stars. In these panels, the CO and \hi  data are shown at a common {\sim}350 pc resolution. The \textit{bottom left} panel shows a map of the total neutral gas surface density adding molecular gas surface density to atomic gas surface density from [Chiang, Sandstrom and Chastenet (2021)](). The white ellipses show 1, 2, 3 \times R_e and the gray dashed ellipse indicates 1 R_{25}. The \textit{lower right} panel shows the azimuthally averaged mass surface density profiles for atomic gas, molecular gas, and stellar mass estimated from the near-infrared. The stellar mass distribution using \textit{HST} imaging is consistent with a nuclear star cluster and an exponential disk  ([Carson, Barth and Seth 2015]()) , while the presented profile suffers from the resolution of the \textit{WISE} data.  Our NOEMA survey covers the inner molecule-dominated region where the \hi  emission is depressed, including the CO-bright center.$
$}$
$\label{fig:bigpicture}$
$\end{figure*}}$
$\newcommand{\FigGMCs}{$
$\begin{figure*}[t!]$
$\centering$
$\includegraphics[width=0.43\textwidth]{figures/MQ_ic342_clouds.png}$
$\includegraphics[width=0.43\textwidth]{figures/MQ_ic342_cprops.png} \\includegraphics[width=0.43\textwidth]{figures/MQ_ic342_cprops_binned.png}$
$\includegraphics[width=0.43\textwidth]{figures/MQ_ic342_beamwise.png}$
$\caption{{Giant molecular clouds or associations in IC 342.} The top left panel shows the location of the molecular clouds that we catalog at 90 pc resolution. The size of each circle corresponds to the deconvolved radius and the color indicates which dynamical environment the cloud is assigned to (red shows interarm clouds, green arm cloud, and blue center clouds). The top right and bottom left panels show the cloud properties in \sigma^2/R vs. \Sigma_{\rm mol} space, in which clouds with a fixed virial parameter follow a diagonal line with a slope of unity in log--log space. \mod{\Sigma_{\rm mol} is the average surface density within the FWHM size of each cloud, \Sigma_{\rm mol} = M_{\rm CO} / (2\pi R^2).} We show the IC 342 clouds for each environment and clouds at the same resolution and sensitivity, but better velocity resolution, from PHANGS--ALMA \citep[][and A. Hughes et al. in preparation]{ROSOLOWSKY21}. The bottom left panel replaces individual clouds with running medians. The bottom right panel adopts a beam-wise approach in which each independent line of sight at a fixed 90 pc resolution supplies a measurement of line width and surface density  ([Sun, Leroy and Schruba 2018](), [Sun, Leroy and Schinnerer 2020]()) . Again we compare the IC 342 points to those from a large sample of PHANGS--ALMA galaxies and here we mark the channel width and approximate sensitivity limit for the IC 342 data with dashed lines \mod{(for a Gaussian CO line, \mathrm{FWHM}=2.35 \sigma, so we always have more than one channel across the FWHM of the emission line)}. The bottom row also indicates the mass-weighted median \mod{(position of the color circles)} and 16{-}84\% range \mod{(span of the color horizontal or vertical line)} for surface densities and line widths. \mod{These refer to the corresponding horizontal or vertical axis, with an arbitrary positioning along the perpendicular direction.} The \textit{mass-weighted} averages tend to lie at higher values than the bulk of the individual measurements because much of the molecular gas mass resides in a few high-mass clouds or associations. All panels tell a consistent story: arm clouds show mildly enhanced surface density and line width compared to inter-arm clouds, and the center shows significant enhancements in surface density and line width. The elevated line widths in the center indicate high virial parameters suggesting clouds with additional contributions to their line widths. The deviations from self-gravity virialization would be even more extreme if we adopted a CO-to-H_2 conversion factor below the Galactic value used to construct these plots. \mod{The black crosses in the right panels show representative error bars for our measurements in IC 342, as explained in the main text.}}$
$\label{fig:clouds}$
$\end{figure*}}$
$\newcommand{\FigSFE}{$
$\begin{figure}$
$\centering$
$\includegraphics[width=0.475\textwidth]{figures/MQ_ic342_t_dep_vs_t_eff.png}$
$\caption{{Star formation efficiency per gravitational free-fall time in 1.5 kpc regions.} Each hexagonal point shows the integrated molecular gas depletion time for a 1.5 kpc diameter region as a function of the mass-weighted average \tau_{\rm ff} (see Eq.~\ref{eq:tau_ff}) for 90 pc resolution clouds in that region. A straight, diagonal line in this space, such as the dark red one, corresponds to a fixed \epsilon_{\rm ff}, which could be expected if density variations represent the primary drivers of depletion time variations. We color the regions by galactocentric radius and calculate the implied \epsilon_{\rm ff} for each region. We estimate a median \epsilon_{\rm ff} of 0.45\% with a 16{-}84\% range of 0.33{-}0.71\%, and we illustrate these with the solid red line and shaded pink region. For the central 1.5 kpc region, we illustrate the effect of switching from our adopted Galactic \alpha_{\rm CO} to a starburst conversion factor. \mod{The black cross shows a representative error bar as explained in the main text. The plotted values can be found in Table~\ref{table:hexagons}.}}$
$\label{fig:eff}$
$\end{figure}}$
$\newcommand{\FigZoom}{$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.95\linewidth]{figures/deprojected-zoom-2}$
$\caption{Zooms of the molecular surface density, peak temperature, and line width towards the galaxy center. The images have been deprojected from the galaxy inclination on the plane of sky, and then converted to kpc using a distance of 3.45 Mpc. \mod{The yellow lines show our spiral mask, while the red and white ellipses delimit the extent of the bar and center environment, respectively (see Appendix~\ref{app:environment} for the newcommandinition of these environment)}.}$
$\label{fig:zoom}$
$\end{figure*}}$
$\newcommand{\FigNIR}{$
$\begin{figure}$
$\centering$
$\includegraphics[width=\linewidth]{figures/irac.png}$
$\caption{\mod{\textit{Spitzer} IRAC 3.6 \mum image showing the newcommandinition of our adopted environments. The inset on the bottom-right highlights the center with a different color stretch.}$
$}$
$\label{fig:NIR}$
$\end{figure}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\Heidelberg}{\label{Heidelberg} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\COOL}{\label{COOL} Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UCSDAA}{\label{UCSDAA} Department of Astronomy \& Astrophysics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} Institut Laue-Langevin, 71 avenue des Martyrs, F-38042 Grenoble, France}$
$\newcommand{\TUM}{\label{TUM} Technical University of Munich, School of Engineering and Design, Department of Aerospace and Geodesy, Chair of Remote Sensing Technology, Arcisstr. 21, 80333 Munich, Germany}$
$\newcommand{\}{mod}$</div>



<div id="title">

# A sensitive, high-resolution, wide-field IRAM NOEMA CO(1-0) survey of the very nearby spiral galaxy IC 342

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06501-b31b1b.svg)](https://arxiv.org/abs/2310.06501)<mark>Appeared on: 2023-10-11</mark> -  _16 pages, 6 figures, accepted for publication in A&A_

</div>
<div id="authors">

M. Querejeta, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark>, <mark>K. Kreckel</mark>, <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** We present a new wide-field $10.75\times10.75$ arcmin $^2$ ( $\approx 11\times11$ kpc $^2$ ), high-resolution ( $\mod{$\theta = 3.6\arcsec \approx 60$ pc}$ ) NOEMA $\chem{CO}{10}$ survey of the very nearby ( $d=3.45$ Mpc) spiral galaxy IC 342. The survey spans out to about $1.5$ effective radii and covers most of the region where molecular gas dominates the cold interstellar medium. We resolved the CO emission into ${>}600$ individual giant molecular clouds and associations. We assessed their properties and found that overall the clouds show approximate virial balance, with typical virial parameters of $\alpha_{\rm vir} = 1{-}2$ . The typical surface density and line width of molecular gas increase from the inter-arm region to the arm $\mod{and bar}$ region, and they reach their highest values in the inner $\mod{kiloparsec}$ of the galaxy ${(median $\Sigma_{\rm mol} \approx 80, 140, 160$, and $1100 M_\odot$ pc$^{-2}$, $\sigma_{\rm CO} \approx 6.6$, $7.6$, $9.7$, and $18.4$ km s$^{-1}$ for inter-arm, arm, bar, and center clouds, respectively)}$ . Clouds in the central part of the galaxy show an enhanced line width relative to their surface densities and evidence of additional sources of dynamical broadening. All of these results agree well with studies of clouds in more distant galaxies at a similar physical resolution. Leveraging our measurements to estimate the density and gravitational free-fall time at $90$ pc resolution, ${averaged on 1.5 kpc hexagonal apertures,}$ we estimate a typical star formation efficiency per free-fall time of $0.45\%$ with a $16{-}84$ \% variation of $0.33{-}0.71\%$ ${among such $1.5$ kpc regions.}$ $\mod{We speculate that bar-driven gas inflow could explain the large gas concentration in the central kiloparsec and the buildup of the massive nuclear star cluster.}$ This wide-area CO map of the closest face-on massive spiral galaxy demonstrates the current mapping power of NOEMA and has many potential applications. The data and products are publicly available.

</div>

<div id="div_fig1">

<img src="tmp_2310.06501/./figures/ic342_tpeak.png" alt="Fig4.1" width="25%"/><img src="tmp_2310.06501/./figures/ic342_mom1.png" alt="Fig4.2" width="25%"/><img src="tmp_2310.06501/./figures/ic342_wide_logtg.png" alt="Fig4.3" width="25%"/><img src="tmp_2310.06501/./figures/ic0342_rprof.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** {NOEMA survey of \chem{CO}{10} emission from IC 342.} The distribution of the \chem{CO}{10} peak intensity at the native angular resolution (_top left_) and the intensity-weighted mean CO velocity at $90$ pc resolution (_top right_) as revealed by our new CO survey. The images show spiral arms, abundant inter-arm emission, and a velocity field that mostly reflects a regularly rotating gas disk. The _bottom_ panels show the molecular gas traced by CO in the context of atomic gas and stars. In these panels, the CO and $\hi$  data are shown at a common ${\sim}350$ pc resolution. The _bottom left_ panel shows a map of the total neutral gas surface density adding molecular gas surface density to atomic gas surface density from [Chiang, Sandstrom and Chastenet (2021)](). The white ellipses show $1, 2, 3 \times R_e$ and the gray dashed ellipse indicates $1 R_{25}$. The _lower right_ panel shows the azimuthally averaged mass surface density profiles for atomic gas, molecular gas, and stellar mass estimated from the near-infrared. The stellar mass distribution using _HST_ imaging is consistent with a nuclear star cluster and an exponential disk  ([Carson, Barth and Seth 2015]()) , while the presented profile suffers from the resolution of the _WISE_ data.  Our NOEMA survey covers the inner molecule-dominated region where the $\hi$  emission is depressed, including the CO-bright center.
 (*fig:bigpicture*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06501/./figures/MQ_ic342_clouds.png" alt="Fig5.1" width="25%"/><img src="tmp_2310.06501/./figures/MQ_ic342_cprops.png" alt="Fig5.2" width="25%"/><img src="tmp_2310.06501/./figures/MQ_ic342_cprops_binned.png" alt="Fig5.3" width="25%"/><img src="tmp_2310.06501/./figures/MQ_ic342_beamwise.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** {Giant molecular clouds or associations in IC 342.} The top left panel shows the location of the molecular clouds that we catalog at $90$ pc resolution. The size of each circle corresponds to the deconvolved radius and the color indicates which dynamical environment the cloud is assigned to (red shows interarm clouds, green arm cloud, and blue center clouds). The top right and bottom left panels show the cloud properties in $\sigma^2/R$ vs. $\Sigma_{\rm mol}$ space, in which clouds with a fixed virial parameter follow a diagonal line with a slope of unity in log--log space. \mod{$\Sigma_{\rm mol}$ is the average surface density within the FWHM size of each cloud, $\Sigma_{\rm mol} = M_{\rm CO} / (2\pi R^2)$.} We show the IC 342 clouds for each environment and clouds at the same resolution and sensitivity, but better velocity resolution, from PHANGS--ALMA \citep[][and A. Hughes et al. in preparation]{ROSOLOWSKY21}. The bottom left panel replaces individual clouds with running medians. The bottom right panel adopts a beam-wise approach in which each independent line of sight at a fixed $90$ pc resolution supplies a measurement of line width and surface density  ([Sun, Leroy and Schruba 2018](), [Sun, Leroy and Schinnerer 2020]()) . Again we compare the IC 342 points to those from a large sample of PHANGS--ALMA galaxies and here we mark the channel width and approximate sensitivity limit for the IC 342 data with dashed lines \mod{(for a Gaussian CO line, $\mathrm{FWHM}=2.35 \sigma$, so we always have more than one channel across the FWHM of the emission line)}. The bottom row also indicates the mass-weighted median \mod{(position of the color circles)} and $16{-}84$\% range \mod{(span of the color horizontal or vertical line)} for surface densities and line widths. \mod{These refer to the corresponding horizontal or vertical axis, with an arbitrary positioning along the perpendicular direction.} The _mass-weighted_ averages tend to lie at higher values than the bulk of the individual measurements because much of the molecular gas mass resides in a few high-mass clouds or associations. All panels tell a consistent story: arm clouds show mildly enhanced surface density and line width compared to inter-arm clouds, and the center shows significant enhancements in surface density and line width. The elevated line widths in the center indicate high virial parameters suggesting clouds with additional contributions to their line widths. The deviations from self-gravity virialization would be even more extreme if we adopted a CO-to-$H_2$ conversion factor below the Galactic value used to construct these plots. \mod{The black crosses in the right panels show representative error bars for our measurements in IC 342, as explained in the main text.} (*fig:clouds*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06501/./figures/MQ_ic342_t_dep_vs_t_eff.png" alt="Fig1" width="100%"/>

**Figure 1. -** {Star formation efficiency per gravitational free-fall time in $1.5$ kpc regions.} Each hexagonal point shows the integrated molecular gas depletion time for a $1.5$ kpc diameter region as a function of the mass-weighted average $\tau_{\rm ff}$(see Eq. \ref{eq:tau_ff}) for $90$ pc resolution clouds in that region. A straight, diagonal line in this space, such as the dark red one, corresponds to a fixed $\epsilon_{\rm ff}$, which could be expected if density variations represent the primary drivers of depletion time variations. We color the regions by galactocentric radius and calculate the implied $\epsilon_{\rm ff}$ for each region. We estimate a median $\epsilon_{\rm ff}$ of $0.45\%$ with a $16{-}84$\% range of $0.33{-}0.71\%$, and we illustrate these with the solid red line and shaded pink region. For the central $1.5$ kpc region, we illustrate the effect of switching from our adopted Galactic $\alpha_{\rm CO}$ to a starburst conversion factor. \mod{The black cross shows a representative error bar as explained in the main text. The plotted values can be found in Table \ref{table:hexagons}.} (*fig:eff*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06501"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\linktodm}{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel}$
$\newcommand{\linktoparam}[2]{\href{\linktodm/sec_dm_main_source_catalogue/ssec_dm_#1.html\##1-#2}{ \texttt{\small#2}\xspace}}$
$\newcommand{\linktotable}[1]{\href{\linktodm/sec_dm_main_source_catalogue/ssec_dm_#1.html}{ \texttt{\small#1}\xspace}}$
$\newcommand{\linktodmfpr}{https://gea.esac.esa.int/archive/documentation/FPR/chap_datamodel}$
$\newcommand{\linktoparamfpr}[2]{\href{\linktodmfpr/sec_dm_focused_product_release/ssec_dm_#1.html\##1-#2}{ \texttt{\small#2}\xspace}}$
$\newcommand{\linktotablefpr}[1]{\href{\linktodmfpr/sec_dm_focused_product_release/ssec_dm_#1.html}{\texttt{ \small#1}\xspace}}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand$</div>



<div id="title">

# $_ Gaia_$ Focused Product Release:\\Sources from Service Interface Function image analysis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06551-b31b1b.svg)](https://arxiv.org/abs/2310.06551)<mark>Appeared on: 2023-10-11</mark> - 

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** $_ Gaia_$ 's readout window strategy is challenged by very dense fields in the sky. Therefore, in addition to standard $_ Gaia_$ observations, full Sky Mapper (SM) images were recorded for nine selected regions in the sky. A new software pipeline exploits these Service Interface Function (SIF) images of crowded fields (CFs),  making use of the availability of the full two-dimensional (2D) information. This new pipeline produced half a million additional $_ Gaia_$ sources in the region of the omega Centauri ( $\omega$ Cen) cluster, which are published with this Focused Product Release. We discuss the dedicated SIF CF data reduction pipeline, validate its data products, and introduce their $_ Gaia_$ archive table. Our aim is to improve the completeness of the $_ Gaia_$ source inventory in a very dense region in the sky, $\omega$ Cen. An adapted version of $_ Gaia_$ 's Source Detection and Image Parameter Determination software located sources in the 2D SIF CF images. These source detections were clustered and assigned to new SIF CF or existing $_ Gaia_$ sources by $_ Gaia_$ 's cross-match software. For the new sources, astrometry was calculated using the Astrometric Global Iterative Solution software, and photometry was obtained in the $_ Gaia_$ DR3 reference system. We validated the results by comparing them to the public $_ Gaia_$ DR3 catalogue and external Hubble Space Telescope data. With this Focused Product Release, 526 587 new sources have been added to the $_ Gaia_$ catalogue in $\omega$ Cen. Apart from positions and brightnesses, the additional catalogue contains parallaxes and proper motions, but no meaningful colour information. While SIF CF source parameters generally have a lower precision than nominal $_ Gaia_$ sources, in the cluster centre they increase the depth of the combined catalogue by three magnitudes and improve the source density by a factor of ten. This first SIF CF data publication already adds great value to the $_ Gaia_$ catalogue. It demonstrates what to expect for the fourth $_ Gaia_$ catalogue, which will contain additional sources for all nine SIF CF regions.

</div>

<div id="div_fig1">

<img src="tmp_2310.06551/./BelliniHST_mags.png" alt="Fig17" width="100%"/>

**Figure 17. -** Comparison of SIF CF data extended with _ Gaia_ DR3 and HST data. Left: Distribution of the combined _ Gaia_ SIF CF and _ Gaia_ DR3 sources (solid dark blue) versus _ Gaia_ G magnitude; of these, sources matched (dashed) and not matched (dash-dotted) to HST; of the latter, not blended  sources (dotted).
        Centre: Distribution of all HST sources (solid teal) versus _ Gaia_ G magnitude; HST sources matched (dashed) and not matched (dash-dotted) to SIF CF sources; Unmatched HST sources potentially resolvable by SIF CF (dotted).
        Right: Fraction of combined SIF CF and _ Gaia_ DR3 sources not matched to HST (dash-dotted dark blue); of these, not HST blends (dotted dark blue); fraction of HST sources not matched to SIF CF (dash-dotted teal); fraction of resolvable, unmatched HST sources (dotted teal) versus (estimated) G magnitude (*fig:bellini*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06551/./dr3_col_terms_before.png" alt="Fig2.1" width="50%"/><img src="tmp_2310.06551/./dr3_mag_terms_before.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Colour and magnitude terms between the SIF CF calibrated magnitudes and the reference magnitudes from _ Gaia_ DR3. The magnitude terms have been corrected for in this FPR. (*fig:phot_colmagterms*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06551/./calibrators.png" alt="Fig5" width="100%"/>

**Figure 5. -** Median G magnitude uncertainty versus G magnitude. G magnitude uncertainty for all calibrators with data density in grey, see the legend and text for a description of the various overlaid curves. (*fig:phot_calibrators*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06551"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

295  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
